In [2]:
import torch 
import torch.nn as nn
import torch.autograd.function as F

In [3]:
class TheModelClass(nn.Module):
    def __init__(self):
        super(TheModelClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [4]:
model = TheModelClass()

In [7]:
for idx, x in enumerate(model.parameters()):
    if idx == 1:
        print(x)
        break


Parameter containing:
tensor([-0.0919, -0.0595, -0.0448, -0.0218,  0.0978,  0.0685],
       requires_grad=True)


In [1]:
a

NameError: name 'a' is not defined

In [1]:
import numpy as np


def k_possibilities(k):
    res = np.random.rand(k)
    _sum = sum(res)
    for i, x in enumerate(res):
        per = x / _sum
        res[i] = per
    return res

In [3]:
k =20
k_possibilities(k)

array([0.04626489, 0.08755396, 0.02541897, 0.02438411, 0.05491319,
       0.08655863, 0.05145549, 0.01541044, 0.05230597, 0.06473681,
       0.04327382, 0.04789705, 0.09333501, 0.00609484, 0.01678263,
       0.01070974, 0.02592164, 0.07231403, 0.0825906 , 0.09207817])

In [6]:
np.random.rand(1)

array([0.94261331])

In [11]:
np.random.multinomial(500, k_possibilities(10))

array([49, 37, 34, 35, 98, 48, 63, 12, 65, 59])

In [15]:
np.random.multinomial(100, [0, 0.5, .5])

array([ 0, 49, 51])